In [314]:
import os
import datetime
import time
import requests
import pandas as pd
import json
import ast

from geopy.geocoders import Nominatim

In [279]:
def convert_date_to_unix(x):
    """
    Convert datetime to unix time in milliseconds.
    """
    dt_obj = datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    dt_obj = int(dt_obj.timestamp() * 1000)
    return dt_obj

In [280]:
def get_city_coordinates(city_name: str):
    """
    Takes city name and returns its latitude and longitude (rounded to 2 digits after dot).
    """ 
    # Initialize Nominatim API (for getting lat and long of the city)
    geolocator = Nominatim(user_agent="MyApp")
    city = geolocator.geocode(city_name)

    latitude = round(city.latitude, 2)
    longitude = round(city.longitude, 2)
    
    return latitude, longitude

# Representing the Target cities 

In [544]:
with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)

In [545]:
for i in target_cities:
    print(i)
    print(len(target_cities[i]))

Seattle
15
EU
17
US
13


In [546]:
from pprint import pprint

pprint(target_cities)

{'EU': {'Amsterdam': [52.37, 4.89],
        'Athina': [37.98, 23.73],
        'Berlin': [52.52, 13.39],
        'Gdansk': [54.37, 18.61],
        'Kraków': [50.06, 19.94],
        'London': [51.51, -0.13],
        'Madrid': [40.42, -3.7],
        'Marseille': [43.3, 5.37],
        'Milano': [45.46, 9.19],
        'München': [48.14, 11.58],
        'Napoli': [40.84, 14.25],
        'Paris': [48.85, 2.35],
        'Sevilla': [37.39, -6.0],
        'Stockholm': [59.33, 18.07],
        'Tallinn': [59.44, 24.75],
        'Varna': [43.21, 27.92],
        'Wien': [48.21, 16.37]},
 'Seattle': {'Bellevue-SE 12th St': [47.60086, -122.1484],
             'DARRINGTON - FIR ST (Darrington High School)': [48.2469,
                                                              -121.6031],
             'KENT - JAMES & CENTRAL': [47.38611, -122.23028],
             'LAKE FOREST PARK TOWNE CENTER': [47.755, -122.2806],
             'MARYSVILLE - 7TH AVE (Marysville Junior High)': [48.05432,
             

In [135]:
# with open("target_cities.json", "w") as json_file:
#     json.dump(target_cities, json_file)

### ALL target cities on the one map

In [547]:
with open('dict_for_map.json') as json_file:
        dict_for_map = json.load(json_file)

In [549]:
# Create a folium map centered on the first location in the list
my_map = folium.Map(location=[42.57, -44.092], zoom_start=3)

for city, coord in dict_for_map.items():
    folium.CircleMarker(
        location=coord,
        popup=city
    ).add_to(my_map)

# Save the map to an HTML file
my_map.save("map_all_target_cities.html")
my_map

# Notes

In [412]:
# for i in target_cities["Seattle"]:
#     a, b = target_cities["Seattle"][i]
#     target_cities["Seattle"][i] = round(a, 5), round(b, 5)

In [414]:
# with open('target_cities.json', "w") as json_file:
#     json.dump(target_cities, json_file)

In [ ]:
# import ast

# string = "(47.1864, -122.4517)"
# t = ast.literal_eval(string)
# t, type(t)

In [ ]:
# seattle_info = seattle_df[["city_name", "latitude", "longitude"]].drop_duplicates()
# for city_name, lat, long in seattle_info.values:
#     target_cities["Seattle"][city_name] = (lat, long)

In [ ]:
# for city in (target_cities["EU"] + target_cities["US"]):
#      dict_for_map[city] = get_city_coordinates(city)

In [ ]:
# for continent in target_cities:
#     for city, coords in target_cities[continent].items():
#         print(city, coords)

# [EEA](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm)
## EEA means European Environmental Agency

In [4]:
def convert_to_daily(df, pollutant: str):
    """
    Returns DataFrame where pollutant column is resampled to days and rounded.
    """
    res_df = df.copy()
    # convert dates in 'time' column
    res_df["date"] = pd.to_datetime(res_df["date"])
    
    # I want data daily, not hourly (mean per each day = 1 datarow per 1 day)
    res_df = res_df.set_index('date')
    res_df = res_df[pollutant].resample('1d').mean().reset_index()
    res_df[pollutant] = res_df[pollutant].fillna(res_df[pollutant].median())
    res_df[pollutant] = res_df[pollutant].apply(lambda x: round(x, 0))
    
    return res_df

In [5]:
def find_fullest_csv(csv_links: list, year: str):
    candidates = [link for link in csv_links if str(year) in link]
    biggest_df = pd.read_csv(candidates[0])
    for link in candidates[1:]:
        _df = pd.read_csv(link)
        if len(biggest_df) < len(_df):
            biggest_df = _df
    return biggest_df

In [6]:
def get_air_quality_from_eea(city_name: str,
                             pollutant: str,
                             start_year: str,
                             end_year: str):
    """
    Takes city name, daterange and returns pandas DataFrame with daily air quality data.
    It parses data by 1-year batches, so please specify years, not dates. (example: "2014", "2022"...)
    
    EEA means European Environmental Agency. So it has data for Europe Union countries ONLY.
    """
    start_of_cell = time.time()
    
    params = {
        'CountryCode': '',
        'CityName': city_name,
        'Pollutant': pollutant.upper(),
        'Year_from': start_year,
        'Year_to': end_year,
        'Station': '',
        'Source': 'All',
        'Samplingpoint': '',
        'Output': 'TEXT',
        'UpdateDate': '',
        'TimeCoverage': 'Year'
    }

    # observations endpoint
    base_url = "https://fme.discomap.eea.europa.eu/fmedatastreaming/AirQualityDownload/AQData_Extract.fmw?"

    response = requests.get(base_url, params=params)

    response.encoding = response.apparent_encoding
    csv_links = response.text.split("\r\n")
    
    res_df = pd.DataFrame()
    target_year = int(start_year)
    
    for year in range(int(start_year), int(end_year) + 1):
        try:
            # find the fullest, the biggest csv file with observations for this particular year
            _df = find_fullest_csv(csv_links, year)
            # append it to res_df
            res_df = pd.concat([res_df, _df])
        except IndexError:
            print(f"!! Missing data for {year} for {city} city.")
            pass
        
    
    pollutant = pollutant.lower()
    if pollutant == "pm2.5":
        pollutant = "pm2_5"
        
    res_df = res_df.rename(columns={
        'DatetimeBegin': 'date',
        'Concentration': pollutant        
    })
    
    # cut timezones info
    res_df['date'] = res_df['date'].apply(lambda x: x[:-6])
    # convert dates in 'time' column
    res_df['date'] = pd.to_datetime(res_df['date'])
    
    res_df = convert_to_daily(res_df, pollutant)
    
    res_df['city_name'] = city_name
    res_df = res_df[['city_name', 'date', pollutant.lower()]]
    
    end_of_cell = time.time()
    
    print(f"Processed {pollutant.upper()} for {city_name} since {start_year} till {end_year}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
    
    return res_df

In [7]:
df_eea_ = get_air_quality_from_eea(
    city_name="Barcelona", pollutant="PM2.5",
    start_year="2013", end_year="2013"
)

df_eea_.head(3)

Processed PM2_5 for Barcelona since 2013 till 2013.
Took 11.16 sec.



,city_name,date,pm2_5
0,Barcelona,2013-01-01,19.0
1,Barcelona,2013-01-02,23.0
2,Barcelona,2013-01-03,28.0


In [553]:
target_cities["EU"]

{'Amsterdam': [52.37, 4.89],
 'Athina': [37.98, 23.73],
 'Berlin': [52.52, 13.39],
 'Gdansk': [54.37, 18.61],
 'Kraków': [50.06, 19.94],
 'London': [51.51, -0.13],
 'Madrid': [40.42, -3.7],
 'Marseille': [43.3, 5.37],
 'Milano': [45.46, 9.19],
 'München': [48.14, 11.58],
 'Napoli': [40.84, 14.25],
 'Paris': [48.85, 2.35],
 'Sevilla': [37.39, -6.0],
 'Stockholm': [59.33, 18.07],
 'Tallinn': [59.44, 24.75],
 'Varna': [43.21, 27.92],
 'Wien': [48.21, 16.37]}

In [10]:
start_of_cell = time.time()

pollutant = "PM2.5" # it will become "pm2_5" anyway
start_year = 2013
end_year = 2023

df_eu = pd.DataFrame()

for city in target_cities["EU"]:
    df_ = get_air_quality_from_eea(
        city_name=city, pollutant=pollutant,
        start_year=start_year, end_year=end_year
    )
    df_eu = pd.concat([df_eu, df_])

end_of_cell = time.time()
print("-" * 64)
print(f"Processed {pollutant.upper()} for EU cities since {start_year} till {end_year}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Amsterdam since 2013 till 2023.
Took 61.22 sec.

Processed PM2_5 for Athina since 2013 till 2023.
Took 25.29 sec.

Processed PM2_5 for Berlin since 2013 till 2023.
Took 42.02 sec.

Processed PM2_5 for Gdansk since 2013 till 2023.
Took 12.21 sec.

Processed PM2_5 for Kraków since 2013 till 2023.
Took 17.68 sec.

Processed PM2_5 for London since 2013 till 2023.
Took 70.08 sec.

Processed PM2_5 for Madrid since 2013 till 2023.
Took 56.7 sec.

Processed PM2_5 for Marseille since 2013 till 2023.
Took 21.62 sec.

Processed PM2_5 for Milano since 2013 till 2023.
Took 22.92 sec.

Processed PM2_5 for München since 2013 till 2023.
Took 48.32 sec.

Processed PM2_5 for Napoli since 2013 till 2023.
Took 48.52 sec.

Processed PM2_5 for Paris since 2013 till 2023.
Took 54.6 sec.

Processed PM2_5 for Sevilla since 2013 till 2023.
Took 9.27 sec.

Processed PM2_5 for Stockholm since 2013 till 2023.
Took 62.78 sec.

Processed PM2_5 for Tallinn since 2013 till 2023.
Took 5.84 sec.

!! 

In [82]:
df_eu

,city_name,date,pm2_5
0,Amsterdam,2013-01-01,14.0
1,Amsterdam,2013-01-02,8.0
2,Amsterdam,2013-01-03,12.0
3,Amsterdam,2013-01-04,12.0
4,Amsterdam,2013-01-05,14.0
...,...,...,...
3748,Wien,2023-04-07,20.0
3749,Wien,2023-04-08,10.0
3750,Wien,2023-04-09,15.0
3751,Wien,2023-04-10,18.0


In [81]:
df_eu.isna().sum().sum()

0

In [142]:
df_eu.to_csv("data/backfill_pm2_5_eu.csv", index=False)

# [USEPA](https://aqs.epa.gov/aqsweb/documents/data_api.html#daily)
## USEPA means United States Environmental Protection Agency
[Manual downloading](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

In [31]:
city_code_dict = {}
pollutant_dict = {
    'CO': '42101',
    'SO2': '42401',
    'NO2': '42602',
    'O3': '44201',
    'PM10': '81102',
    'PM2.5': '88101'
}

def get_city_code(city_name: str):
    "Encodes city name to be used later for data parsing using USEPA."
    if city_code_dict:
        city_full = [i for i in city_code_dict.keys() if city_name in i][0]
        return city_code_dict[city_full]
    else:
        params = {
            "email": "test@aqs.api",
            "key": "test"
        }
        response = requests.get("https://aqs.epa.gov/data/api/list/cbsas?", params)
        response_json = response.json()
        data = response_json["Data"]
        for item in data:
            city_code_dict[item['value_represented']] = item['code']
        
        return get_city_code(city_name)

In [32]:
def make_date_intervals(start_date, end_date):
    start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    date_intervals = []
    for year in range(start_dt.year, end_dt.year + 1):
        year_start = datetime.datetime(year, 1, 1)
        year_end = datetime.datetime(year, 12, 31)
        interval_start = max(start_dt, year_start)
        interval_end = min(end_dt, year_end)
        if interval_start < interval_end:
            date_intervals.append((interval_start.strftime('%Y%m%d'), interval_end.strftime('%Y%m%d')))
    return date_intervals


In [44]:
def get_air_quality_from_usepa(city_name: str,
                               pollutant: str,
                               start_date: str,
                               end_date: str):
    """
    Takes city name, daterange and returns pandas DataFrame with daily air quality data.
    
    USEPA means United States Environmental Protection Agency. So it has data for US ONLY.
    """
    start_of_cell = time.time()
        
    res_df = pd.DataFrame()
    
#     # to print 'Success' log only once.
#     was = False
    for start_date_, end_date_ in make_date_intervals(start_date, end_date):
        params = {
            "email": "test@aqs.api",
            "key": "test",
            "param": pollutant_dict[pollutant.upper().replace("_", ".")], # encoded pollutant 
            "bdate": start_date_,
            "edate": end_date_,
            "cbsa": get_city_code(city_name) # Core-based statistical area
        }

        # observations endpoint
        base_url = "https://aqs.epa.gov/data/api/dailyData/byCBSA?" 

        response = requests.get(base_url, params=params)
        response_json = response.json()
        # if not was:
        #     print(response_json["Header"][0]["status"])
        #     was = True
        
        df_ = pd.DataFrame(response_json["Data"])
        
        pollutant = pollutant.lower()
        
        if pollutant == "pm2.5":
            pollutant = "pm2_5"
        df_ = df_.rename(columns={
            'date_local': 'date',
            'arithmetic_mean': pollutant        
        })

        # convert dates in 'date' column
        df_['date'] = pd.to_datetime(df_['date'])
        df_['city_name'] = city_name    
       
        df_ = df_[['city_name', 'date', pollutant]]

        res_df = pd.concat([res_df, df_])
    
    # there are duplicated rows (several records for the same day and station). get rid of it.
    res_df = res_df.groupby(['date', 'city_name'], as_index=False)[pollutant].mean()
    res_df[pollutant] = round(res_df[pollutant], 1)  
    
    end_of_cell = time.time()
    print(f"Processed {pollutant.upper()} for {city_name} since {start_date} till {end_date}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
    
    return res_df

In [72]:
df_usepa_ = get_air_quality_from_usepa(city_name="Albuquerque", pollutant="PM2.5",
                                       start_date="2013-07-01", end_date="2014-01-01")

Processed PM2_5 for Albuquerque since 2013-07-01 till 2014-01-01.
Took 2.16 sec.



In [73]:
df_usepa_

,date,city_name,pm2_5
0,2013-07-01,Albuquerque,4.4
1,2013-07-02,Albuquerque,12.4
2,2013-07-03,Albuquerque,5.6
3,2013-07-04,Albuquerque,12.3
4,2013-07-05,Albuquerque,11.2
...,...,...,...
179,2013-12-27,Albuquerque,8.6
180,2013-12-28,Albuquerque,14.7
181,2013-12-29,Albuquerque,9.3
182,2013-12-30,Albuquerque,9.3


In [75]:
target_cities["US"]

['Albuquerque',
 'Atlanta',
 'Chicago',
 'Columbus',
 'Dallas',
 'Denver',
 'Houston',
 'Los Angeles',
 'New York',
 'Phoenix-Mesa',
 'Salt Lake City',
 'San Francisco',
 'Tampa']

In [77]:
start_of_cell = time.time()

pollutant = "PM2.5"

start_date = "2013-01-01"
end_date = "2023-01-01"

df_us = pd.DataFrame()

for city in target_cities["US"]:
    df_ = get_air_quality_from_usepa(
        city_name=city, pollutant=pollutant,
        start_date=start_date, end_date=end_date
        )
    df_us = pd.concat([df_us, df_])
    
end_of_cell = time.time()
print("-" * 64)
print(f"Processed {pollutant.upper()} for US cities since {start_year} till {end_year}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Albuquerque since 2013-01-01 till 2023-01-01.
Took 63.28 sec.

Processed PM2_5 for Atlanta since 2013-01-01 till 2023-01-01.
Took 53.23 sec.

Processed PM2_5 for Chicago since 2013-01-01 till 2023-01-01.
Took 120.86 sec.

Processed PM2_5 for Columbus since 2013-01-01 till 2023-01-01.
Took 42.43 sec.

Processed PM2_5 for Dallas since 2013-01-01 till 2023-01-01.
Took 44.47 sec.

Processed PM2_5 for Denver since 2013-01-01 till 2023-01-01.
Took 72.45 sec.

Processed PM2_5 for Houston since 2013-01-01 till 2023-01-01.
Took 54.07 sec.

Processed PM2_5 for Los Angeles since 2013-01-01 till 2023-01-01.
Took 87.88 sec.

Processed PM2_5 for New York since 2013-01-01 till 2023-01-01.
Took 138.85 sec.

Processed PM2_5 for Phoenix-Mesa since 2013-01-01 till 2023-01-01.
Took 121.5 sec.

Processed PM2_5 for Salt Lake City since 2013-01-01 till 2023-01-01.
Took 109.5 sec.

Processed PM2_5 for San Francisco since 2013-01-01 till 2023-01-01.
Took 100.42 sec.

Processed PM2_5 for Tam

In [78]:
df_us

,date,city_name,pm2_5
0,2013-01-01,Albuquerque,6.8
1,2013-01-02,Albuquerque,8.4
2,2013-01-03,Albuquerque,7.8
3,2013-01-04,Albuquerque,9.2
4,2013-01-05,Albuquerque,12.2
...,...,...,...
3609,2022-12-27,Tampa,6.8
3610,2022-12-28,Tampa,6.9
3611,2022-12-29,Tampa,5.2
3612,2022-12-30,Tampa,5.4


In [79]:
df_us.isna().sum().sum()

0

In [143]:
df_us.to_csv("data/backfill_pm2_5_us.csv", index=False)

# Processing special city - `Seattle`.
### We need different stations across the Seattle. 

I downloaded daily `PM2.5` data manually from [here](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

In [554]:
df_seattle = pd.DataFrame()

for year in range(2013, 2023 + 1):
    df_ = pd.read_csv(f"data/seattle_pm25_{year}.csv")
    df_seattle = pd.concat([df_seattle, df_])

df_seattle = df_seattle.reset_index(drop=True)

df_seattle.shape

(67901, 20)

In [555]:
df_seattle.tail(2)

,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration,UNITS,DAILY_AQI_VALUE,Site Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
67899,04/02/2023,AirNow,530611007,5,4.8,ug/m3 LC,20,MARYSVILLE - 7TH AVE (Marysville Junior High),1,100.0,88101,PM2.5 - Local Conditions,42660,"Seattle-Tacoma-Bellevue, WA",53,Washington,61,Snohomish,48.054315,-122.171529
67900,04/03/2023,AirNow,530611007,5,4.8,ug/m3 LC,20,MARYSVILLE - 7TH AVE (Marysville Junior High),1,100.0,88101,PM2.5 - Local Conditions,42660,"Seattle-Tacoma-Bellevue, WA",53,Washington,61,Snohomish,48.054315,-122.171529


In [556]:
df_seattle = df_seattle.rename(columns={
    'Daily Mean PM2.5 Concentration': 'pm2_5',
    'Date': 'date',
    'SITE_LATITUDE': 'latitude',
    'SITE_LONGITUDE': 'longitude',
    'Site Name': 'city_name'
})[['city_name', 'date', 'pm2_5', 'latitude', 'longitude']]

In [557]:
df_seattle = df_seattle.drop_duplicates(subset=['date', 'city_name'])

In [558]:
df_seattle.city_name.value_counts()

NORTH BEND - NORTH BEND WAY                                       3705
TACOMA - L STREET                                                 3696
SEATTLE - BEACON HILL                                             3691
MARYSVILLE - 7TH AVE (Marysville Junior High)                     3648
DARRINGTON - FIR ST (Darrington High School)                      3614
SEATTLE - SOUTH PARK #2                                           3577
TACOMA - ALEXANDER AVE                                            3569
KENT - JAMES & CENTRAL                                            3556
SEATTLE - DUWAMISH                                                3439
Seattle-10th & Weller                                             3097
LAKE FOREST PARK TOWNE CENTER                                     2999
PUYALLUP - 128TH ST                                               2700
Tacoma-S 36th St                                                  2574
Bellevue-SE 12th St                                               2172
LYNNWO

In [559]:
df_seattle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57227 entries, 0 to 67807
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city_name  56502 non-null  object 
 1   date       57227 non-null  object 
 2   pm2_5      57227 non-null  float64
 3   latitude   57227 non-null  float64
 4   longitude  57227 non-null  float64
dtypes: float64(3), object(2)
memory usage: 2.6+ MB


In [560]:
df_seattle['date'] = pd.to_datetime(df_seattle['date'])

# Group the dataframe by city_name and count the number of rows for each group
counts = df_seattle.groupby('city_name').size()

# Filter the stations that have more than 1000 rows
filtered_counts = counts[counts > 1000]

# Get the city_names of the filtered stations
filtered_sites = filtered_counts.index

# Filter the dataframe to only include rows from the filtered stations
filtered_df = df_seattle[df_seattle['city_name'].isin(filtered_sites)]

# Filter the dataframe to only include rows with a date newer than 2022-08-01
filtered_df = filtered_df[filtered_df['date'] > '2022-08-01']

# Get the city_names of the filtered stations that have at least one row newer than 2022-08-01
newer_sites = filtered_df['city_name'].unique().tolist()

# Create a list of stations that meet both criteria
sites_to_leave = [site for site in filtered_sites if site in newer_sites]

# Print the station list
print(sites_to_leave)

['Bellevue-SE 12th St', 'DARRINGTON - FIR ST (Darrington High School)', 'KENT - JAMES & CENTRAL', 'LAKE FOREST PARK TOWNE CENTER', 'MARYSVILLE - 7TH AVE (Marysville Junior High)', 'NORTH BEND - NORTH BEND WAY', 'SEATTLE - BEACON HILL', 'SEATTLE - DUWAMISH', 'SEATTLE - SOUTH PARK #2', 'Seattle-10th & Weller', 'TACOMA - ALEXANDER AVE', 'TACOMA - L STREET', 'Tacoma-S 36th St', 'Tukwila Allentown', 'Tulalip-Totem Beach Rd']


## Considering data quantity and the freshness of data for each site, I decided to cut off some sites:

In [561]:
df_seattle = df_seattle[df_seattle.city_name.isin(sites_to_leave)].reset_index(drop=True)

In [562]:
df_seattle = df_seattle.dropna()

In [563]:
df_seattle.shape

(46479, 5)

In [564]:
# # lets rename these sites so we could later concat this df with other cities data

# df_seattle.city_name= df_seattle.city_name.apply(lambda x: "Seattle - " + x)

In [565]:
df_seattle.city_name.value_counts()

NORTH BEND - NORTH BEND WAY                      3705
TACOMA - L STREET                                3696
SEATTLE - BEACON HILL                            3691
MARYSVILLE - 7TH AVE (Marysville Junior High)    3648
DARRINGTON - FIR ST (Darrington High School)     3614
SEATTLE - SOUTH PARK #2                          3577
TACOMA - ALEXANDER AVE                           3569
KENT - JAMES & CENTRAL                           3556
SEATTLE - DUWAMISH                               3439
Seattle-10th & Weller                            3097
LAKE FOREST PARK TOWNE CENTER                    2999
Tacoma-S 36th St                                 2574
Bellevue-SE 12th St                              2172
Tukwila Allentown                                2074
Tulalip-Totem Beach Rd                           1068
Name: city_name, dtype: int64

In [566]:
# df_seattle['latitude'] = df_seattle['latitude'].apply(lambda x: round(x, 5))
# df_seattle['longitude'] = df_seattle['longitude'].apply(lambda x: round(x, 5))

# df_seattle["coordinates"] = tuple(zip(df_seattle["latitude"], df_seattle["longitude"]))
df_seattle = df_seattle.drop(columns=["latitude", "longitude"])

In [567]:
df_seattle.sample(1)

,city_name,date,pm2_5
15770,Bellevue-SE 12th St,2017-07-01,4.6


In [568]:
df_seattle.to_csv("data/backfill_pm2_5_seattle.csv", index=False)

In [569]:
df_seattle = pd.read_csv("data/backfill_pm2_5_seattle.csv")

In [570]:
df_seattle.head(3)

,city_name,date,pm2_5
0,NORTH BEND - NORTH BEND WAY,2013-01-01,4.7
1,NORTH BEND - NORTH BEND WAY,2013-01-02,2.8
2,NORTH BEND - NORTH BEND WAY,2013-01-03,3.2


In [571]:
df_seattle.shape

(46479, 3)

# Weather data from [open meteo](https://open-meteo.com/en/docs)

- Maximum Temperature (2 m)
- Minimum Temperature (2 m)
- Precipitation Sum
- Rain Sum
- Snowfall Sum
- Precipitation Hours
- Maximum Wind Speed (10 m)
- Maximum Wind Gusts (10 m)
- Dominant Wind Direction (10 m)


In [426]:
def get_weather_data_from_open_meteo(city_name: str,
                                     start_date: str,
                                     end_date: str,
                                     coordinates: list = None,
                                     forecast: bool = False):
    """
    Takes [city name OR coordinates] and returns pandas DataFrame with weather data.
    
    Examples of arguments:
        coordinates=(47.755, -122.2806), start_date="2023-01-01"
    """
    start_of_cell = time.time()
    
    if coordinates:
        latitude, longitude = coordinates
    else:
        latitude, longitude = get_city_coordinates(city_name=city_name)
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'daily': ["temperature_2m_max", "temperature_2m_min",
                  "precipitation_sum", "rain_sum", "snowfall_sum",
                  "precipitation_hours", "windspeed_10m_max",
                  "windgusts_10m_max", "winddirection_10m_dominant"],
        'start_date': start_date,
        'end_date': end_date,
        'timezone': "Europe/London"
    }
    
    if forecast:
        # historical forecast endpoint
        base_url = 'https://api.open-meteo.com/v1/forecast' 
    else:
        # historical observations endpoint
        base_url = 'https://archive-api.open-meteo.com/v1/archive' 
        
    response = requests.get(base_url, params=params)

    response_json = response.json()    
    res_df = pd.DataFrame(response_json["daily"])
    
    res_df["city_name"] = city_name
    
    # rename columns
    res_df = res_df.rename(columns={
        "time": "date",
        "temperature_2m_max": "temperature_max",
        "temperature_2m_min": "temperature_min",
        "windspeed_10m_max": "wind_speed_max",
        "winddirection_10m_dominant": "wind_direction_dominant",
        "windgusts_10m_max": "wind_gusts_max"
    })
    
    # change columns order
    res_df = res_df[
        ['city_name', 'date', 'temperature_max', 'temperature_min',
         'precipitation_sum', 'rain_sum', 'snowfall_sum',
         'precipitation_hours', 'wind_speed_max',
         'wind_gusts_max', 'wind_direction_dominant']
    ]
    
    # convert dates in 'date' column
    res_df["date"] = pd.to_datetime(res_df["date"])
    
#     # create 'unix' columns
#     res_df["unix_time"] = res_df["base_time"].apply(convert_date_to_unix)
    end_of_cell = time.time()
    print(f"Parsed weather for {city_name} since {start_date} till {end_date}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
        
    return res_df

In [427]:
get_weather_data_from_open_meteo(city_name="some site in Seattle", 
                                 coordinates=(47.755, -122.2806),
                                 start_date="2023-01-01", end_date="2023-01-01")

Parsed weather for some site in Seattle since 2023-01-01 till 2023-01-01.
Took 0.35 sec.



,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
0,some site in Seattle,2023-01-01,8.1,3.9,2.2,2.2,0.0,10.0,7.8,14.8,140


In [434]:
get_weather_data_from_open_meteo(city_name="Phoenix-Mesa", start_date="2023-01-01", end_date="2023-01-01", forecast=True)

Parsed weather for Phoenix-Mesa since 2023-01-01 till 2023-01-01.
Took 0.54 sec.



,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
0,Phoenix-Mesa,2023-01-01,17.3,11.6,21.6,32.4,0.0,7.0,26.4,39.6,129


In [429]:
today = datetime.date.today() # datetime object

day7next = str(today + datetime.timedelta(7))
day7ago = str(today - datetime.timedelta(7))
tomorrow = str(today + datetime.timedelta(1))

In [528]:
# weather data for ALL cities (and Seattle sites)
df_weather = pd.DataFrame()

In [533]:
# Initialize an empty list to store the 2nd level values (city names)
cities = []

for key in target_cities:
    for city in target_cities[key]:
        cities.append(city)

In [534]:
start_of_cell = time.time()

coordinates = None

param_tuples = [("2013-01-01", day7ago, False), (day7ago, str(today), True)]

for params in param_tuples:
    start_date, end_date, forecast = params
    for city_name in cities: 
        if city_name in target_cities["Seattle"]:
            coordinates = target_cities["Seattle"][city_name]

        df_ = get_weather_data_from_open_meteo(city_name=city_name,
                                               coordinates=coordinates,
                                               start_date=start_date,
                                               end_date=end_date,
                                               forecast=forecast) # firstly I am parsing historical weather data
                                                                  # and then forecasts till today
        df_weather = pd.concat([df_weather, df_]).reset_index(drop=True)



end_of_cell = time.time()
print("-" * 64)
print(f"Parsed historical weather data for EU and US cities up to {str(today)}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Parsed weather for Amsterdam since 2013-01-01 till 2023-04-06.
Took 0.96 sec.

Parsed weather for Athina since 2013-01-01 till 2023-04-06.
Took 0.91 sec.

Parsed weather for Berlin since 2013-01-01 till 2023-04-06.
Took 0.92 sec.

Parsed weather for Gdansk since 2013-01-01 till 2023-04-06.
Took 0.87 sec.

Parsed weather for Kraków since 2013-01-01 till 2023-04-06.
Took 0.84 sec.

Parsed weather for London since 2013-01-01 till 2023-04-06.
Took 0.88 sec.

Parsed weather for Madrid since 2013-01-01 till 2023-04-06.
Took 0.9 sec.

Parsed weather for Marseille since 2013-01-01 till 2023-04-06.
Took 0.82 sec.

Parsed weather for Milano since 2013-01-01 till 2023-04-06.
Took 0.93 sec.

Parsed weather for München since 2013-01-01 till 2023-04-06.
Took 0.89 sec.

Parsed weather for Napoli since 2013-01-01 till 2023-04-06.
Took 0.88 sec.

Parsed weather for Paris since 2013-01-01 till 2023-04-06.
Took 0.87 sec.

Parsed weather for Sevilla since 2013-01-01 till 2023-04-06.
Took 0.83 sec.

Parsed

KeyboardInterrupt: 

In [506]:
df_weather.date = pd.to_datetime(df_weather.date)
df_weather.date = df_weather.date.astype(str)

In [537]:
df_weather = df_weather.drop_duplicates(["city_name", "date"])

In [510]:
df_weather[['city_name', 'date']].groupby('city_name').count()

,date
city_name,
Albuquerque,3755
Amsterdam,3755
Athina,3755
Atlanta,3755
Berlin,3755
Chicago,3755
Columbus,3755
Dallas,3755
Denver,3755


In [511]:
df_weather.to_csv("data/backfill_weather.csv", index=False)